<a href="https://colab.research.google.com/github/githubofjay/FCC-MachineLearning-Course/blob/main/Copy_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
# try:
#   # %tensorflow_version only exists in Colab.
#   !pip install tf-nightly
# except Exception:
#   pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.15.0


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-02-27 15:46:29--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.007s  

2024-02-27 15:46:29 (50.3 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2024-02-27 15:46:29--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [3]:
df_train = pd.read_csv(train_file_path, sep = "\t", header = None, names = ['y', 'x'])
df_test = pd.read_csv(test_file_path, sep = "\t", header = None, names = ['y', 'x'])

y_train = df_train['y'].astype('category').cat.codes
y_test = df_test['y'].astype('category').cat.codes
y_train[:5]

0    0
1    0
2    0
3    0
4    0
dtype: int8

In [4]:
import nltk
import re
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [5]:
stopwords_eng = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()

def clean_txt(txt):
  txt = re.sub(r'([^\s\w])+', ' ', txt)
  txt = txt.lower()
  return txt

X_train = df_train['x'].apply(lambda x : clean_txt(x))
X_train[:5]

0    ahhhh just woken up had a bad dream about u th...
1                             you can never do nothing
2    now u sound like manky scouse boy steve like  ...
3    mum say we wan to go then go  then she can shu...
4    never y lei  i v lazy  got wat  dat day ü send...
Name: x, dtype: object

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

#Keep top 1000 frequently occuring words
max_words = 1000

#Cut off the words after seeing 500 words in each document
max_len = 500

t = Tokenizer(num_words = max_words)
t.fit_on_texts(X_train)

In [7]:
sequences = t.texts_to_sequences(X_train)
sequences[:5]

seqs_matrix = sequence.pad_sequences(sequences, maxlen = max_len)
seqs_matrix[:5]

array([[  0,   0,   0, ...,  47,  13,  12],
       [  0,   0,   0, ..., 279,  31, 343],
       [  0,   0,   0, ...,  43, 435, 531],
       [  0,   0,   0, ...,  25, 344,  99],
       [  0,   0,   0, ..., 200, 143,  79]], dtype=int32)

In [8]:
from keras import models
from keras import layers

i = tf.keras.layers.Input(shape=[max_len])
x = tf.keras.layers.Embedding(max_words, 50)(i)
x = tf.keras.layers.LSTM(64)(x)

x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='relu')(x)

model = tf.keras.models.Model(inputs=i, outputs=x)
model.compile(
    loss='binary_crossentropy',
    optimizer='RMSprop',
    metrics=['accuracy']
)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 500)]             0         
                                                                 
 embedding (Embedding)       (None, 500, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 dense (Dense)               (None, 256)               16640     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 96337 (376.32 KB)
Trainable params: 96337 (376.

In [9]:
r = model.fit(seqs_matrix, y_train,
              batch_size=128, epochs=10,
              validation_split=0.2,
              callbacks=[tf.keras.callbacks.EarlyStopping(
                  monitor='val_loss', min_delta=0.0001)])

Epoch 1/10
27/27 [==============================] - 36s 1s/step - loss: 0.3942 - accuracy: 0.8762 - val_loss: 0.1514 - val_accuracy: 0.9438
Epoch 2/10
27/27 [==============================] - 23s 848ms/step - loss: 0.1175 - accuracy: 0.9620 - val_loss: 0.1022 - val_accuracy: 0.9904
Epoch 3/10
27/27 [==============================] - 21s 784ms/step - loss: 0.0611 - accuracy: 0.9853 - val_loss: 0.0926 - val_accuracy: 0.9916
Epoch 4/10
27/27 [==============================] - 21s 794ms/step - loss: 0.0461 - accuracy: 0.9895 - val_loss: 0.0871 - val_accuracy: 0.9892
Epoch 5/10
27/27 [==============================] - 24s 884ms/step - loss: 0.0409 - accuracy: 0.9919 - val_loss: 0.1069 - val_accuracy: 0.9844


In [10]:
def preprocessing(X):
  x = X.apply(lambda x:clean_txt(x))
  x = t.texts_to_sequences(x)
  return sequence.pad_sequences(x, maxlen = max_len)

s = model.evaluate(preprocessing(df_test['x']), y_test)

44/44 [==============================] - 3s 59ms/step - loss: 0.0594 - accuracy: 0.9842


In [11]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  p = model.predict(preprocessing(pd.Series([pred_text])))[0]

  return (p[0], ("ham" if p<0.5 else "spam"))

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 493ms/step
(0.0, 'ham')


In [12]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

1/1 [==============================] - 0s 34ms/step
You passed the challenge. Great job!
